In [207]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import bs4
import pandas as pd
import numpy as np

In [208]:
keyword = '처빌'
driver = webdriver.Chrome(r'C:\Users\YSM\Downloads\chromedriver_win32\chromedriver.exe')

i = 1

title_list = []
price_list = []
review_list = []
buy_list = []


while True:    
    driver.get('https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery={}&pagingIndex={}&pagingSize=80&productSet=total&query={}&sort=rel&timestamp=&viewType=list'.format(keyword, i, keyword))
    time.sleep(0.5)
    i += 1

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)


    item_source = bs4.BeautifulSoup(driver.page_source, 'lxml')
    item_list = item_source.select('div.basicList_info_area__17Xyo')

    if not item_list:
        break

    else:
        for item in item_list:
            title = item.select_one('a').text.replace('\n', '')
            price = item.select_one('span.price_num__2WUXn').text
            ReviewAndBuy = item.select('a.basicList_etc__2uAYO > em')
            try:
                review = ReviewAndBuy[0].text + '개'
            except IndexError:
                review = '없음'

            try:
                buy = ReviewAndBuy[1].text + '개'
            except IndexError:
                buy = '없음'

            title_list.append(title)
            price_list.append(price)
            review_list.append(review)
            buy_list.append(buy)

df = pd.DataFrame({'상품명':title_list, '가격':price_list, '리뷰수':review_list, '구매건수':buy_list})

In [209]:
df

,상품명,가격,리뷰수,구매건수
0,허브다섯메 국내재배 신선한 챠빌 처빌 10g~,"1,500원",100개,없음
1,처빌 생잎 100g 허브 향신료 가락시장 직송,"6,000원",없음,없음
2,처빌 생잎 1팩(약10g) 가락시장 직송,"1,000원",15개,176개
3,처빌 1팩 10g / 50g 허브 향신료,"2,500원",31개,48개
4,허브 처빌 차빌 쳐빌 생잎 10g 50g 100g Chervil,"2,500원",1개,없음
...,...,...,...,...
246,요즘엔 트렌드 D-956 처빌 메신저백/슬링백/라이더백/바디백/캐주얼백/남성세컨백/...,"24,900원",없음,없음
247,요즘엔 트렌드 D-956 처빌 메신저백/슬링백/라이더백/바디백/크로스백/캐주얼백/남...,"24,900원",없음,없음
248,GT[소망농원]처빌씨앗(약750립)/150립 5봉 묶음/다년생/차빌/전초는 식용 및...,"16,500원",없음,없음
249,GT[소망농원]처빌씨앗(약750립)/150립 5봉 묶음/다년생/차빌/전초는 식용 및...,"16,500원",없음,없음


In [210]:
ext = df.상품명.str.extract('(\d+g|\d+kg|\d+KG)')
    
for i in range(len(ext)):
    if 'kg' in str(ext[0][i]):
        temp = ext[0][i].split('kg')
        ext[0][i] = int(temp[0]) * 1000
    elif 'Kg' in str(ext[0][i]):
        temp = ext[0][i].split('Kg')
        ext[0][i] = int(temp[0]) * 1000
    elif 'KG' in str(ext[0][i]):
        temp = ext[0][i].split('KG')
        ext[0][i] = int(temp[0]) * 1000
    elif 'g' in str(ext[0][i]):
        temp = ext[0][i].split('g')
        ext[0][i] = int(temp[0])

In [211]:
price = df.가격
for i in range(len(price)):
    if ',' in price[i]:
        b = price[i].split(',')
        c = b[1].split('원')
        d = b[0] + c[0]
        df['가격'][i] = int(d)
    else:
        c = price[i].split('원')
        df['가격'][i] = int(c[0])

In [212]:
df

,상품명,가격,리뷰수,구매건수
0,허브다섯메 국내재배 신선한 챠빌 처빌 10g~,1500,100개,없음
1,처빌 생잎 100g 허브 향신료 가락시장 직송,6000,없음,없음
2,처빌 생잎 1팩(약10g) 가락시장 직송,1000,15개,176개
3,처빌 1팩 10g / 50g 허브 향신료,2500,31개,48개
4,허브 처빌 차빌 쳐빌 생잎 10g 50g 100g Chervil,2500,1개,없음
...,...,...,...,...
246,요즘엔 트렌드 D-956 처빌 메신저백/슬링백/라이더백/바디백/캐주얼백/남성세컨백/...,24900,없음,없음
247,요즘엔 트렌드 D-956 처빌 메신저백/슬링백/라이더백/바디백/크로스백/캐주얼백/남...,24900,없음,없음
248,GT[소망농원]처빌씨앗(약750립)/150립 5봉 묶음/다년생/차빌/전초는 식용 및...,16500,없음,없음
249,GT[소망농원]처빌씨앗(약750립)/150립 5봉 묶음/다년생/차빌/전초는 식용 및...,16500,없음,없음


In [213]:
notNull = []

checkNull = ext.notnull()

for i in range(len(checkNull)):
    if checkNull[0][i] == True:
        notNull.append(i)

In [214]:
weight = df.loc[notNull]

In [215]:
weight['무게(g)'] = ext[0]

In [216]:
weight['g당 가격'] = weight['가격'] / weight['무게(g)']
weight

,상품명,가격,리뷰수,구매건수,무게(g),g당 가격
0,허브다섯메 국내재배 신선한 챠빌 처빌 10g~,1500,100개,없음,10,150
1,처빌 생잎 100g 허브 향신료 가락시장 직송,6000,없음,없음,100,60
2,처빌 생잎 1팩(약10g) 가락시장 직송,1000,15개,176개,10,100
3,처빌 1팩 10g / 50g 허브 향신료,2500,31개,48개,10,250
4,허브 처빌 차빌 쳐빌 생잎 10g 50g 100g Chervil,2500,1개,없음,10,250
...,...,...,...,...,...,...
203,처빌 0.2g/봄파종 허브씨앗/파종시기별/허브씨앗,2000,없음,없음,2,1000
204,일년생허브씨/씨앗/처빌 0.2g,2200,없음,없음,2,1100
205,차로먹는허브씨/씨앗/처빌 0.2g,2200,없음,없음,2,1100
216,허브씨/봄파종허브씨/씨앗/처빌 0.2g,2200,없음,없음,2,1100
